In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from tensorflow import keras

def prepare_data(seq,num):
  x=[]
  y=[]
  for i in range(0,(len(seq)-num),1):
    
    input_ = seq[i:i+num]
    output  = seq[i+num]
    
    x.append(input_)
    y.append(output)
    
  return np.array(x), np.array(y)

data=pd.read_csv('../Utilities\csv\metrics_2024-09-03\metrics.csv')
print(data.head)
cpu_usage = data['value'].values

num=20
#first 10 mins of traffic
sample = cpu_usage[:num]

x,y= prepare_data(cpu_usage,num)
print(len(x))

ind = int(0.9 * len(x))
x_tr = x[:ind]
y_tr = y[:ind]
x_val=x[ind:]
y_val=y[ind:]

#normalize the inputs
x_scaler= StandardScaler()
x_tr = x_scaler.fit_transform(x_tr)
x_val= x_scaler.transform(x_val)

#reshaping the output for normalization
y_tr=y_tr.reshape(len(y_tr),1)
y_val=y_val.reshape(len(y_val),1)

#normalize the output
y_scaler=StandardScaler()
y_tr = y_scaler.fit_transform(y_tr)[:,0]
y_val = y_scaler.transform(y_val)[:,0]

#reshaping input data
x_tr= x_tr.reshape(x_tr.shape[0],x_tr.shape[1],1)
x_val= x_val.reshape(x_val.shape[0],x_val.shape[1],1)
print(x_tr.shape)

# define model
model =  Sequential()
model.add(Conv1D(64, 3, padding='same', activation='relu',input_shape=(num,1)))
model.add(Conv1D(32, 5, padding='same', activation='relu',input_shape=(num,1)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='linear'))
model.summary()
# Define the optimizer and loss:
model.compile(loss='mse',optimizer='adam')
# Define the callback to save the best model during the training
mc = ModelCheckpoint('best_model.hdf5', monitor='val_loss', verbose=1,
         save_best_only=True, mode='min')
# Train the model for 30 epochs with batch size of 32:
history=model.fit(x_tr, y_tr ,epochs=30, batch_size=32, validation_data=(x_val,y_val),
            callbacks=[mc])
model.load_weights('best_model.hdf5')
mse = model.evaluate(x_val,y_val)
print("Mean Square Error:",mse)

<bound method NDFrame.head of                timestamp         value
0    03/09/2024 11:00:00  1.165827e-06
1    03/09/2024 11:00:30  1.189352e-06
2    03/09/2024 11:01:00  1.143872e-06
3    03/09/2024 11:01:30  1.141328e-06
4    03/09/2024 11:02:00  0.000000e+00
..                   ...           ...
353  03/09/2024 14:32:30  2.971772e-07
354  03/09/2024 14:33:00  2.837418e-07
355  03/09/2024 14:33:30  1.520919e-07
356  03/09/2024 14:34:00  1.643452e-07
357  03/09/2024 14:34:30  0.000000e+00

[358 rows x 2 columns]>
338
(304, 20, 1)
Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 20, 64)            256       
                                                                 
 conv1d_13 (Conv1D)          (None, 20, 32)            10272     
                                                                 
 flatten_6 (Flatten)         (None, 640) 

c:\Users\brand\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - 0s 22ms/step - loss: 0.2339 - val_loss: 3.9666e-05
Epoch 3/30
 8/10 [=======================>......] - ETA: 0s - loss: 0.1081    
Epoch 3: val_loss did not improve from 0.00004
10/10 [==============================] - 0s 17ms/step - loss: 0.0964 - val_loss: 1.5993e-04
Epoch 4/30
 6/10 [=================>............] - ETA: 0s - loss: 0.0583
Epoch 4: val_loss did not improve from 0.00004
10/10 [==============================] - 0s 15ms/step - loss: 0.0440 - val_loss: 8.0552e-05
Epoch 5/30
 7/10 [====================>.........] - ETA: 0s - loss: 0.0265
Epoch 5: val_loss did not improve from 0.00004
10/10 [==============================] - 0s 19ms/step - loss: 0.0266 - val_loss: 4.7911e-05
Epoch 6/30
 6/10 [=================>............] - ETA: 0s - loss: 0.0090
Epoch 6: val_loss did not improve from 0.00004
10/10 [==============================] - 0s 20ms/step - loss: 0.0129 - val_loss: 6.4625e-04
Epoch 7/30
 6/10 [=================>............